In [1]:
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras import backend
import os
import torch
import torch.utils.data
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from math import sin, cos
from PIL import ImageDraw, Image
import cv2
sys.path.append(os.path.abspath("./Residual-Attention-Network/"))
from model.residual_attention_network import ResidualAttentionModel_92 as ResidualAttentionModel
import cv2

Using TensorFlow backend.


In [2]:
train = pd.read_csv('./train.csv')
# k is camera instrinsic matrix
k = np.array([[2304.5479, 0,  1686.2379],
           [0, 2305.8757, 1354.9849],
           [0, 0, 1]], dtype = np.float32)

In [3]:
# convert euler angle to rotation matrix
def euler_to_Rot(yaw, pitch, roll):
    Y = np.array([[cos(yaw), 0, sin(yaw)],
                  [0, 1, 0],
                  [-sin(yaw), 0, cos(yaw)]])
    P = np.array([[1, 0, 0],
                  [0, cos(pitch), -sin(pitch)],
                  [0, sin(pitch), cos(pitch)]])
    R = np.array([[cos(roll), -sin(roll), 0],
                  [sin(roll), cos(roll), 0],
                  [0, 0, 1]])
    return np.dot(Y, np.dot(P, R))

In [4]:
def draw_line(image, points):
    color = (255, 0, 0)
    cv2.line(image, tuple(points[1][:2]), tuple(points[2][:2]), color, 16)
    cv2.line(image, tuple(points[1][:2]), tuple(points[4][:2]), color, 16)

    cv2.line(image, tuple(points[1][:2]), tuple(points[5][:2]), color, 16)
    cv2.line(image, tuple(points[2][:2]), tuple(points[3][:2]), color, 16)
    cv2.line(image, tuple(points[2][:2]), tuple(points[6][:2]), color, 16)
    cv2.line(image, tuple(points[3][:2]), tuple(points[4][:2]), color, 16)
    cv2.line(image, tuple(points[3][:2]), tuple(points[7][:2]), color, 16)

    cv2.line(image, tuple(points[4][:2]), tuple(points[8][:2]), color, 16)
    cv2.line(image, tuple(points[5][:2]), tuple(points[8][:2]), color, 16)

    cv2.line(image, tuple(points[5][:2]), tuple(points[6][:2]), color, 16)
    cv2.line(image, tuple(points[6][:2]), tuple(points[7][:2]), color, 16)
    cv2.line(image, tuple(points[7][:2]), tuple(points[8][:2]), color, 16)
    return image


def draw_points(image, points):
    image = np.array(image)
    for (p_x, p_y) in points:
        # print("p_x, p_y", p_x, p_y)
        cv2.circle(image, (p_x, p_y), 5, (255, 0, 0), -1)
    return image

In [5]:
# image coordinate to world coordinate
def img_cor_2_world_cor():
    x_img, y_img, z_img = img_cor_points[0]
    xc, yc, zc = x_img*z_img, y_img*z_img, z_img
    p_cam = np.array([xc, yc, zc])
    xw, yw, zw = np.dot(np.linalg.inv(k), p_cam)
    print(xw, yw, zw)
    print(x, y, z)

In [6]:
# Now begin the initialization of dictionary for image training set

In [7]:
X_MAX = 3383
Y_MAX = 2709
class stage2Dataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, mode = 'train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train = pd.read_csv(csv_file)
        if mode == 'train':
            self.train = self.train[:int(self.train.shape[0]*0.7)]
        else:
            self.train = self.train[int(self.train.shape[0]*0.7):]
    def __len__(self):
        return self.train.shape[0]

    def __getitem__(self, idx):
        x_l = 1.02 # augment this
        y_l = 0.80 # augment this
        z_l = 2.31 # augment this
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = train.loc[idx]['ImageId']
        pred_string = train.loc[idx]['PredictionString']
        img = plt.imread('./train_images/' + img_name + '.jpg').astype(float)/255
        mg_y, mg_x = np.meshgrid(np.linspace(0, 1, img.shape[0]), np.linspace(0, 1, img.shape[1]), indexing = 'ij')
        #process prediction string
        items = pred_string.split(' ')
        model_types, yaws, pitches, rolls, xs, ys, zs = [items[i::7] for i in range(7)]
        sample = []
        #iterate over each vehcicle
        for yaw, pitch, roll, x, y, z in zip(yaws, pitches, rolls, xs, ys, zs):
            yaw, pitch, roll, x, y, z = [float(x) for x in [yaw, pitch, roll, x, y, z]]
            img_x, img_y, img_z = np.dot(k, [x, y, z])
            target = [yaw, pitch, roll, img_x/10000, img_y/10000, img_z] #result to regress to
            # I think the pitch and yaw should be exchanged
            yaw, pitch, roll = -pitch, -yaw, -roll
            Rt = np.eye(4)
            t = np.array([x, y, z])
            Rt[:3, 3] = t
            Rt[:3, :3] = euler_to_Rot(yaw, pitch, roll).T
            Rt = Rt[:3, :]
            P = np.array([[0, 0, 0, 1],
                          [x_l, y_l, -z_l, 1],
                          [x_l, y_l, z_l, 1],
                          [-x_l, y_l, z_l, 1],
                          [-x_l, y_l, -z_l, 1],
                          [x_l, -y_l, -z_l, 1],
                          [x_l, -y_l, z_l, 1],
                          [-x_l, -y_l, z_l, 1],
                          [-x_l, -y_l, -z_l, 1]]).T

            # call this function before change the astype
            img_cor_points = np.dot(k, np.dot(Rt, P)) # calculate 8 corners and center in terms of camera coordinate
            img_cor_points = img_cor_points.T
            img_cor_points[:, 0] /= img_cor_points[:, 2] # project onto 2d camera plane
            img_cor_points[:, 1] /= img_cor_points[:, 2]
            x_max = int(np.amax(img_cor_points[:,0])) # calculate 2d box
            if x_max>X_MAX:
                x_max = X_MAX
            x_min = int(np.amin(img_cor_points[:,0]))
            if x_min<0:
                x_min = 0
            y_max = int(np.amax(img_cor_points[:,1]))
            if y_max>Y_MAX:
                y_max = Y_MAX
            y_min = int(np.amin(img_cor_points[:,1])) 
            if y_min<0:
                y_min = 0
            # Crop image
            img_crop = cv2.resize(img[y_min:y_max, x_min:x_max], (224, 224))
            mg_x_crop = cv2.resize(mg_x[y_min:y_max, x_min:x_max], (224, 224))[..., np.newaxis]
            mg_y_crop = cv2.resize(mg_y[y_min:y_max, x_min:x_max], (224, 224))[..., np.newaxis]
            input_tensor = np.concatenate((img_crop, mg_y_crop, mg_x_crop), axis = 2).transpose((2, 0, 1))
            sample.append((input_tensor, target))
        return sample

In [8]:
stage2set = stage2Dataset('./train.csv')
trainloader = torch.utils.data.DataLoader(stage2set, batch_size=1, shuffle = True)

In [9]:
net = ResidualAttentionModel()
for batch_idx, sample in enumerate(trainloader):
    X = torch.zeros([len(sample), 5, 224, 224])
    Y = torch.zeros([len(sample), 6])
    for sample_idx, (input_tensor, target) in enumerate(sample):
        X[sample_idx] = input_tensor[0]
        Y[sample_idx] = target[0]
        print(target)
    break

TypeError: new() received an invalid combination of arguments - got (float, int, int, int), but expected one of:
 * (torch.device device)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, torch.device device)
 * (object data, torch.device device)
